In [2]:
### Optcuts
import os

meshdir = "./data/coseg_raw/vases_processed_lr"
savedir = "./data/coseg_vases"
basestr = "python optcuts_process.py "

filecount = 0
max_array = 1000
tmpcount = 0

f = open(f"./slurm/vases_preprocess_{filecount}", 'w')
for meshfile in os.listdir(meshdir):
    if not meshfile.endswith(".obj"):
        continue
    meshname = meshfile.replace(".obj", "")

    for i in range(20):
        if tmpcount >= max_array:
            f.close()
            filecount += 1
            tmpcount = 0
            f = open(f"./slurm/vases_preprocess_{filecount}", 'w')

        savepath = os.path.join(savedir, meshname + f"_{i}")
        expstr = basestr + f"--i {i} --savepath {savepath} --meshpath {os.path.join(meshdir, meshfile)}"
        f.write(expstr + "\n")

        tmpcount += 1
f.close()

In [2]:
### CSD Lighting tuning
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 10000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp \
--csd_maxweight 0.5 --lossdistortion confvar --amips --normalizeinit"

textures = ["checkerboard.jpeg"]
models = ['can_soup', 'manzana', 'sofa_1']

captions = {'bob_floatie': {'blackpepperbrick': ['a rubber duck made of grey brick', 'a statue of a rubber duck made of brick'],
                            'bricktextureinternet': ['a rubber duck made of red brick', 'a statue of a rubber duck made of red brick'],
                            'chainmail': ['a rubber duck made of chainmail', 'a rubber duck wearing chainmail'],
                            'checkerboard': ['a rubber duck with a red and black checkerboard pattern',
                                             'a rubber duck textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a rubber duck covered in colorful dragon scales',
                                            'a rubber duck with skin made of dragon scales'],
                            'sideup': ['a rubber duck covered in this side up stickers'],
                            'sisalweave': ['a rubber duck made of woven rope'],
                            'treebark': ['a rubber duck made of tree bark', 'a rubber duck covered in tree bark']},
            'can_soup': {'blackpepperbrick': ['a soup can made of grey brick', 'a statue of a soup can made of brick'],
                            'bricktextureinternet': ['a soup can made of red brick', 'a statue of a soup can made of red brick'],
                            'chainmail': ['a soup can made of chainmail', 'a soup can wearing chainmail'],
                            'checkerboard': ['a soup can with a red and black checkerboard pattern',
                                             'a soup can textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a soup can covered in colorful dragon scales',
                                            'a soup can with skin made of dragon scales'],
                            'sideup': ['a soup can covered in this side up stickers'],
                            'sisalweave': ['a soup can made of woven rope'],
                            'treebark': ['a soup can made of tree bark', 'a soup can covered in tree bark']},
            'laptop_for_music_production': {'blackpepperbrick': ['a laptop made of grey brick', 'a statue of a laptop made of brick'],
                            'bricktextureinternet': ['a laptop made of red brick', 'a statue of a laptop made of red brick'],
                            'chainmail': ['a laptop made of chainmail', 'a laptop wearing chainmail'],
                            'checkerboard': ['a laptop with a red and black checkerboard pattern',
                                             'a laptop textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a laptop covered in colorful dragon scales',
                                            'a laptop in a cover made of dragon scales'],
                            'sideup': ['a laptop covered in this side up stickers'],
                            'sisalweave': ['a laptop made of woven rope'],
                            'treebark': ['a laptop made of tree bark', 'a laptop covered in tree bark']},
            'manzana': {'blackpepperbrick': ['an apple made of grey brick', 'a statue of an apple made of brick'],
                            'bricktextureinternet': ['an apple made of red brick', 'a statue of an apple made of red brick'],
                            'chainmail': ['an apple made of chainmail', 'an apple wearing chainmail'],
                            'checkerboard': ['an apple with a red and black checkerboard pattern',
                                             'an apple textured with a red and black checkerboard pattern'],
                            'dragonscale': ['an apple covered in colorful dragon scales',
                                            'an apple with skin made of dragon scales'],
                            'sideup': ['an apple covered in this side up stickers'],
                            'sisalweave': ['an apple made of woven rope'],
                            'treebark': ['an apple made of tree bark', 'an apple covered in tree bark']},
            'sofa_1': {'blackpepperbrick': ['a sofa made of grey brick', 'a statue of a sofa made of brick'],
                            'bricktextureinternet': ['a sofa made of red brick', 'a statue of a sofa made of red brick'],
                            'chainmail': ['a sofa made of chainmail', 'a sofa wearing chainmail'],
                            'checkerboard': ['a sofa with a red and black checkerboard pattern',
                                             'a sofa textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a sofa covered in colorful dragon scales',
                                            'a sofa with a cover made of dragon scales'],
                            'sideup': ['a sofa covered in this side up stickers'],
                            'sisalweave': ['a sofa made of woven rope'],
                            'treebark': ['a sofa made of tree bark', 'a sofa covered in tree bark']},
        'barrel': {'blackpepperbrick': ["a barrel made of stone brick", "a barrel made out of grey brick",
                                            "a statue of a barrel made of stone bricks"],
                       'bricktextureinternet': ['a barrel made of brick', 'a barrel made of red brick'],
                       'chainmail': ['a barrel made of chainmail', 'a barrel covered in a chainmail mesh'],
                       'checkerboard': ['a barrel made of black and red checkerboard', 'a barrel covered in a checkerboard pattern'],
                       'dragonscale': ['a barrel made of dragon scales', 'a barrel made of colorful dragon scales',
                                       'a barrel covered in dragon scales', 'a barrel covered in colorful dragon scales'],
                       'sideup': ['a barrel covered in this side up stickers'],
                       'sisalweave': ['a barrel made of rope', 'a barrel made of rope in a sisal weave pattern'],
                       'stainedwood': ['a barrel made of polished hardwood flooring',
                                       'a barrel covered in polished wood panels',
                                       'a barrel made of stained wood panels'],
                       'treebark': ['a barrel made of tree bark', 'a barrel covered in tree bark'],
                       'concrete': ['a stone statue of a barrel', 'a barrel made of concrete', 'a barrel made of stone'],},
            'hippie_shiba': {'blackpepperbrick': ["a statue of a shiba dog made out of grey brick",
                                            "a statue of a shiba dog made of stone bricks"],
                       'bricktextureinternet': ['a statue of a shiba dog made of brick', 'a statue of a shiba dog made of red brick'],
                       'chainmail': ['a statue of a shiba dog made of chainmail', 'a shiba dog covered in a chainmail mesh'],
                       'checkerboard': ['a statue of a shiba dog painted in black and red checkerboard pattern',
                                        'a shiba dog covered in a black and red checkerboard pattern'],
                       'dragonscale': ['a shiba dog with skin made of dragon scales', 'a shiba dog with skin made of colorful dragon scales',
                                       'a statue of a shiba dog covered in colorful dragon scales'],
                       'sideup': ['a shiba dog covered in this side up stickers',
                                  'a statue of a shiba dog made of upwards-facing red arrows and stickers saying this side up'],
                       'sisalweave': ['a statue of a shiba dog made of rope', 'a shiba dog doll made of rope in a sisal weave pattern'],
                       'stainedwood': ['a statue of a shiba dog made of stained wood',
                                       'a statue of a shiba dog covered in stained wood',
                                       ],
                       'treebark': ['a statue of a shiba dog made of tree bark', 'a shiba dog with skin made of tree bark'],
                       'concrete': ['a stone statue of a shiba dog', 'a concrete statue of a shiba dog', 'a shiba dog made of stone']
                       },
            }

jsons = [model + "_slim.json" for model in models]

# First exp is cylinder sds
f = open("./slurm/nopt_csd", 'w')

randlights = [True, False]
nviews = [1, 5, 10]
csd_weights = [0, 0.5]

for texture in textures:
    for model in models:
        for rand in randlights:
            for nview in nviews:
                for weight in csd_weights:
                    json = model + "_slim.json"
                    texturename = texture.split('.')[0]
                    captionlist = captions[model][texturename]

                    for caption in captionlist:
                        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                        expstr += f" --sdscaption {caption} --nviews {nview}"
                        expstr += f" --sdsloss csd --csd_minweight {weight}"

                        if rand:
                            expstr += " --randomlights"

                        expstr += f" --expname csdexp_{model}_{texturename}_CAPTION_{caption.replace(' ', '_')}_RAND_{rand}_NVIEW_{nview}_WEIGHT_{weight}"
                        f.write(expstr + "\n")
f.close()

In [1]:
### Controlnet tuning (depth shit)
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp \
--csd_maxweight 0.5 --lossdistortion confvar --amips --lowmemory --normalizeinit"

textures = ["checkerboard.jpeg"]
models = ['can_soup', 'manzana', 'sofa_1']

captions = {'bob_floatie': {'blackpepperbrick': ['a rubber duck made of grey brick', 'a statue of a rubber duck made of brick'],
                            'bricktextureinternet': ['a rubber duck made of red brick', 'a statue of a rubber duck made of red brick'],
                            'chainmail': ['a rubber duck made of chainmail', 'a rubber duck wearing chainmail'],
                            'checkerboard': ['a rubber duck with a red and black checkerboard pattern',
                                             'a rubber duck textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a rubber duck covered in colorful dragon scales',
                                            'a rubber duck with skin made of dragon scales'],
                            'sideup': ['a rubber duck covered in this side up stickers'],
                            'sisalweave': ['a rubber duck made of woven rope'],
                            'treebark': ['a rubber duck made of tree bark', 'a rubber duck covered in tree bark']},
            'can_soup': {'blackpepperbrick': ['a soup can made of grey brick', 'a statue of a soup can made of brick'],
                            'bricktextureinternet': ['a soup can made of red brick', 'a statue of a soup can made of red brick'],
                            'chainmail': ['a soup can made of chainmail', 'a soup can wearing chainmail'],
                            'checkerboard': ['a soup can with a red and black checkerboard pattern',
                                             'a soup can textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a soup can covered in colorful dragon scales',
                                            'a soup can with skin made of dragon scales'],
                            'sideup': ['a soup can covered in this side up stickers'],
                            'sisalweave': ['a soup can made of woven rope'],
                            'treebark': ['a soup can made of tree bark', 'a soup can covered in tree bark']},
            'laptop_for_music_production': {'blackpepperbrick': ['a laptop made of grey brick', 'a statue of a laptop made of brick'],
                            'bricktextureinternet': ['a laptop made of red brick', 'a statue of a laptop made of red brick'],
                            'chainmail': ['a laptop made of chainmail', 'a laptop wearing chainmail'],
                            'checkerboard': ['a laptop with a red and black checkerboard pattern',
                                             'a laptop textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a laptop covered in colorful dragon scales',
                                            'a laptop in a cover made of dragon scales'],
                            'sideup': ['a laptop covered in this side up stickers'],
                            'sisalweave': ['a laptop made of woven rope'],
                            'treebark': ['a laptop made of tree bark', 'a laptop covered in tree bark']},
            'manzana': {'blackpepperbrick': ['an apple made of grey brick', 'a statue of an apple made of brick'],
                            'bricktextureinternet': ['an apple made of red brick', 'a statue of an apple made of red brick'],
                            'chainmail': ['an apple made of chainmail', 'an apple wearing chainmail'],
                            'checkerboard': ['an apple with a red and black checkerboard pattern',
                                             'an apple textured with a red and black checkerboard pattern'],
                            'dragonscale': ['an apple covered in colorful dragon scales',
                                            'an apple with skin made of dragon scales'],
                            'sideup': ['an apple covered in this side up stickers'],
                            'sisalweave': ['an apple made of woven rope'],
                            'treebark': ['an apple made of tree bark', 'an apple covered in tree bark']},
            'sofa_1': {'blackpepperbrick': ['a sofa made of grey brick', 'a statue of a sofa made of brick'],
                            'bricktextureinternet': ['a sofa made of red brick', 'a statue of a sofa made of red brick'],
                            'chainmail': ['a sofa made of chainmail', 'a sofa wearing chainmail'],
                            'checkerboard': ['a sofa with a red and black checkerboard pattern',
                                             'a sofa textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a sofa covered in colorful dragon scales',
                                            'a sofa with a cover made of dragon scales'],
                            'sideup': ['a sofa covered in this side up stickers'],
                            'sisalweave': ['a sofa made of woven rope'],
                            'treebark': ['a sofa made of tree bark', 'a sofa covered in tree bark']},
        'barrel': {'blackpepperbrick': ["a barrel made of stone brick", "a barrel made out of grey brick",
                                            "a statue of a barrel made of stone bricks"],
                       'bricktextureinternet': ['a barrel made of brick', 'a barrel made of red brick'],
                       'chainmail': ['a barrel made of chainmail', 'a barrel covered in a chainmail mesh'],
                       'checkerboard': ['a barrel made of black and red checkerboard', 'a barrel covered in a checkerboard pattern'],
                       'dragonscale': ['a barrel made of dragon scales', 'a barrel made of colorful dragon scales',
                                       'a barrel covered in dragon scales', 'a barrel covered in colorful dragon scales'],
                       'sideup': ['a barrel covered in this side up stickers'],
                       'sisalweave': ['a barrel made of rope', 'a barrel made of rope in a sisal weave pattern'],
                       'stainedwood': ['a barrel made of polished hardwood flooring',
                                       'a barrel covered in polished wood panels',
                                       'a barrel made of stained wood panels'],
                       'treebark': ['a barrel made of tree bark', 'a barrel covered in tree bark'],
                       'concrete': ['a stone statue of a barrel', 'a barrel made of concrete', 'a barrel made of stone'],},
            'hippie_shiba': {'blackpepperbrick': ["a statue of a shiba dog made out of grey brick",
                                            "a statue of a shiba dog made of stone bricks"],
                       'bricktextureinternet': ['a statue of a shiba dog made of brick', 'a statue of a shiba dog made of red brick'],
                       'chainmail': ['a statue of a shiba dog made of chainmail', 'a shiba dog covered in a chainmail mesh'],
                       'checkerboard': ['a statue of a shiba dog painted in black and red checkerboard pattern',
                                        'a shiba dog covered in a black and red checkerboard pattern'],
                       'dragonscale': ['a shiba dog with skin made of dragon scales', 'a shiba dog with skin made of colorful dragon scales',
                                       'a statue of a shiba dog covered in colorful dragon scales'],
                       'sideup': ['a shiba dog covered in this side up stickers',
                                  'a statue of a shiba dog made of upwards-facing red arrows and stickers saying this side up'],
                       'sisalweave': ['a statue of a shiba dog made of rope', 'a shiba dog doll made of rope in a sisal weave pattern'],
                       'stainedwood': ['a statue of a shiba dog made of stained wood',
                                       'a statue of a shiba dog covered in stained wood',
                                       ],
                       'treebark': ['a statue of a shiba dog made of tree bark', 'a shiba dog with skin made of tree bark'],
                       'concrete': ['a stone statue of a shiba dog', 'a concrete statue of a shiba dog', 'a shiba dog made of stone']
                       },
            }

jsons = [model + "_slim.json" for model in models]

# First exp is cylinder sds
f = open("./slurm/nopt_controlnet", 'w')

conditions = ['depth', 'canny']
specular = [True, False]

for texture in textures:
    for model in models:
        for spec in specular:
            json = model + "_slim.json"
            texturename = texture.split('.')[0]
            captionlist = captions[model][texturename]

            for caption in captionlist:
                for condition in conditions:
                    expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                    expstr += f" --sdscaption {caption}"

                    expstr += f" --sdsloss controlnet csd --controlcondition {condition}"

                    if spec:
                        expstr += " --specular"

                    expstr += f" --expname control_{model}_{texturename}_CAPTION_{caption.replace(' ', '_')}_CONDITION_{condition}_SPECULAR_{spec}"
                    f.write(expstr + "\n")
f.close()

In [3]:
### Compare influence of controlnet + csd min/max weights
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp \
--csd_maxweight 0.5 --lossdistortion confvar --amips --lowmemory --normalizeinit"

textures = ["checkerboard.jpeg"]
models = ['can_soup', 'manzana', 'sofa_1']

captions = {'bob_floatie': {'blackpepperbrick': ['a rubber duck made of grey brick', 'a statue of a rubber duck made of brick'],
                            'bricktextureinternet': ['a rubber duck made of red brick', 'a statue of a rubber duck made of red brick'],
                            'chainmail': ['a rubber duck made of chainmail', 'a rubber duck wearing chainmail'],
                            'checkerboard': ['a rubber duck with a red and black checkerboard pattern',
                                             'a rubber duck textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a rubber duck covered in colorful dragon scales',
                                            'a rubber duck with skin made of dragon scales'],
                            'sideup': ['a rubber duck covered in this side up stickers'],
                            'sisalweave': ['a rubber duck made of woven rope'],
                            'treebark': ['a rubber duck made of tree bark', 'a rubber duck covered in tree bark']},
            'can_soup': {'blackpepperbrick': ['a soup can made of grey brick', 'a statue of a soup can made of brick'],
                            'bricktextureinternet': ['a soup can made of red brick', 'a statue of a soup can made of red brick'],
                            'chainmail': ['a soup can made of chainmail', 'a soup can wearing chainmail'],
                            'checkerboard': ['a soup can with a red and black checkerboard pattern',
                                             'a soup can textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a soup can covered in colorful dragon scales',
                                            'a soup can with skin made of dragon scales'],
                            'sideup': ['a soup can covered in this side up stickers'],
                            'sisalweave': ['a soup can made of woven rope'],
                            'treebark': ['a soup can made of tree bark', 'a soup can covered in tree bark']},
            'laptop_for_music_production': {'blackpepperbrick': ['a laptop made of grey brick', 'a statue of a laptop made of brick'],
                            'bricktextureinternet': ['a laptop made of red brick', 'a statue of a laptop made of red brick'],
                            'chainmail': ['a laptop made of chainmail', 'a laptop wearing chainmail'],
                            'checkerboard': ['a laptop with a red and black checkerboard pattern',
                                             'a laptop textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a laptop covered in colorful dragon scales',
                                            'a laptop in a cover made of dragon scales'],
                            'sideup': ['a laptop covered in this side up stickers'],
                            'sisalweave': ['a laptop made of woven rope'],
                            'treebark': ['a laptop made of tree bark', 'a laptop covered in tree bark']},
            'manzana': {'blackpepperbrick': ['an apple made of grey brick', 'a statue of an apple made of brick'],
                            'bricktextureinternet': ['an apple made of red brick', 'a statue of an apple made of red brick'],
                            'chainmail': ['an apple made of chainmail', 'an apple wearing chainmail'],
                            'checkerboard': ['an apple with a red and black checkerboard pattern',
                                             'an apple textured with a red and black checkerboard pattern'],
                            'dragonscale': ['an apple covered in colorful dragon scales',
                                            'an apple with skin made of dragon scales'],
                            'sideup': ['an apple covered in this side up stickers'],
                            'sisalweave': ['an apple made of woven rope'],
                            'treebark': ['an apple made of tree bark', 'an apple covered in tree bark']},
            'sofa_1': {'blackpepperbrick': ['a sofa made of grey brick', 'a statue of a sofa made of brick'],
                            'bricktextureinternet': ['a sofa made of red brick', 'a statue of a sofa made of red brick'],
                            'chainmail': ['a sofa made of chainmail', 'a sofa wearing chainmail'],
                            'checkerboard': ['a sofa with a red and black checkerboard pattern',
                                             'a sofa textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a sofa covered in colorful dragon scales',
                                            'a sofa with a cover made of dragon scales'],
                            'sideup': ['a sofa covered in this side up stickers'],
                            'sisalweave': ['a sofa made of woven rope'],
                            'treebark': ['a sofa made of tree bark', 'a sofa covered in tree bark']},
        'barrel': {'blackpepperbrick': ["a barrel made of stone brick", "a barrel made out of grey brick",
                                            "a statue of a barrel made of stone bricks"],
                       'bricktextureinternet': ['a barrel made of brick', 'a barrel made of red brick'],
                       'chainmail': ['a barrel made of chainmail', 'a barrel covered in a chainmail mesh'],
                       'checkerboard': ['a barrel made of black and red checkerboard', 'a barrel covered in a checkerboard pattern'],
                       'dragonscale': ['a barrel made of dragon scales', 'a barrel made of colorful dragon scales',
                                       'a barrel covered in dragon scales', 'a barrel covered in colorful dragon scales'],
                       'sideup': ['a barrel covered in this side up stickers'],
                       'sisalweave': ['a barrel made of rope', 'a barrel made of rope in a sisal weave pattern'],
                       'stainedwood': ['a barrel made of polished hardwood flooring',
                                       'a barrel covered in polished wood panels',
                                       'a barrel made of stained wood panels'],
                       'treebark': ['a barrel made of tree bark', 'a barrel covered in tree bark'],
                       'concrete': ['a stone statue of a barrel', 'a barrel made of concrete', 'a barrel made of stone'],},
            'hippie_shiba': {'blackpepperbrick': ["a statue of a shiba dog made out of grey brick",
                                            "a statue of a shiba dog made of stone bricks"],
                       'bricktextureinternet': ['a statue of a shiba dog made of brick', 'a statue of a shiba dog made of red brick'],
                       'chainmail': ['a statue of a shiba dog made of chainmail', 'a shiba dog covered in a chainmail mesh'],
                       'checkerboard': ['a statue of a shiba dog painted in black and red checkerboard pattern',
                                        'a shiba dog covered in a black and red checkerboard pattern'],
                       'dragonscale': ['a shiba dog with skin made of dragon scales', 'a shiba dog with skin made of colorful dragon scales',
                                       'a statue of a shiba dog covered in colorful dragon scales'],
                       'sideup': ['a shiba dog covered in this side up stickers',
                                  'a statue of a shiba dog made of upwards-facing red arrows and stickers saying this side up'],
                       'sisalweave': ['a statue of a shiba dog made of rope', 'a shiba dog doll made of rope in a sisal weave pattern'],
                       'stainedwood': ['a statue of a shiba dog made of stained wood',
                                       'a statue of a shiba dog covered in stained wood',
                                       ],
                       'treebark': ['a statue of a shiba dog made of tree bark', 'a shiba dog with skin made of tree bark'],
                       'concrete': ['a stone statue of a shiba dog', 'a concrete statue of a shiba dog', 'a shiba dog made of stone']
                       },
            }

jsons = [model + "_slim.json" for model in models]

controls = [False, True]
csd_weights = [0, 0.5]

# First exp is cylinder sds
f = open("./slurm/nopt_compare", 'w')

for texture in textures:
    for model in models:
        json = model + "_slim.json"
        texturename = texture.split('.')[0]
        captionlist = captions[model][texturename]

        for caption in captionlist:
            for control in controls:
                for weight in csd_weights:
                    expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                    expstr += f" --sdscaption {caption} --csd_minweight {weight}"

                    if control:
                        expstr += f" --sdsloss controlnet csd"
                    else:
                        expstr += f" --sdsloss csd"

                    expstr += f" --expname compare_{model}_{texturename}_CAPTION_{caption.replace(' ', '_')}_CONTROL_{control}_CSDWEIGHT_{weight}"
                    f.write(expstr + "\n")
f.close()

In [1]:
### Prompt engineering + controlnet SDS + CSD for parameterization benchmark meshes
# NOTE: try for bob_floatie, can_soup, laptop_for_music_production, manzana, sofa_1
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --sdsloss controlnet csd --spweight sigmoid --arch mlp \
--csd_maxweight 0.5 --lossdistortion confvar --amips --lowmemory"

textures = ["blackpepperbrick.jpg",
            "bricktextureinternet.jpeg",
            "checkerboard.jpeg", "dragonscale.jpeg", "sideup.png",
            "treebark.jpg"]
models = ["bob_floatie", "can_soup", "laptop_for_music_production", "manzana", "sofa_1"]

params = ['', ', 35 mm', ', 4k', ', award-winning', ', unreal engine']
captions = {'bob_floatie': {'blackpepperbrick': ['a rubber duck made of grey brick', 'a statue of a rubber duck made of brick'],
                            'bricktextureinternet': ['a rubber duck made of red brick', 'a statue of a rubber duck made of red brick'],
                            'chainmail': ['a rubber duck made of chainmail', 'a rubber duck wearing chainmail'],
                            'checkerboard': ['a rubber duck with a red and black checkerboard pattern',
                                             'a rubber duck textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a rubber duck covered in colorful dragon scales',
                                            'a rubber duck with skin made of dragon scales'],
                            'sideup': ['a rubber duck covered in upwards-facing red arrows saying this side up'],
                            'sisalweave': ['a rubber duck made of woven rope'],
                            'treebark': ['a rubber duck made of tree bark', 'a rubber duck covered in tree bark']},
            'can_soup': {'blackpepperbrick': ['a soup can made of grey brick', 'a statue of a soup can made of brick'],
                            'bricktextureinternet': ['a soup can made of red brick', 'a statue of a soup can made of red brick'],
                            'chainmail': ['a soup can made of chainmail', 'a soup can wearing chainmail'],
                            'checkerboard': ['a soup can with a red and black checkerboard pattern',
                                             'a soup can textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a soup can covered in colorful dragon scales',
                                            'a soup can with skin made of dragon scales'],
                            'sideup': ['a soup can covered in upwards-facing red arrows saying this side up'],
                            'sisalweave': ['a soup can made of woven rope'],
                            'treebark': ['a soup can made of tree bark', 'a soup can covered in tree bark']},
            'laptop_for_music_production': {'blackpepperbrick': ['a laptop made of grey brick', 'a statue of a laptop made of brick'],
                            'bricktextureinternet': ['a laptop made of red brick', 'a statue of a laptop made of red brick'],
                            'chainmail': ['a laptop made of chainmail', 'a laptop wearing chainmail'],
                            'checkerboard': ['a laptop with a red and black checkerboard pattern',
                                             'a laptop textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a laptop covered in colorful dragon scales',
                                            'a laptop in a cover made of dragon scales'],
                            'sideup': ['a laptop covered in upwards-facing red arrow stickers saying this side up'],
                            'sisalweave': ['a laptop made of woven rope'],
                            'treebark': ['a laptop made of tree bark', 'a laptop covered in tree bark']},
            'manzana': {'blackpepperbrick': ['an apple made of grey brick', 'a statue of an apple made of brick'],
                            'bricktextureinternet': ['an apple made of red brick', 'a statue of an apple made of red brick'],
                            'chainmail': ['an apple made of chainmail', 'an apple wearing chainmail'],
                            'checkerboard': ['an apple with a red and black checkerboard pattern',
                                             'an apple textured with a red and black checkerboard pattern'],
                            'dragonscale': ['an apple covered in colorful dragon scales',
                                            'an apple with skin made of dragon scales'],
                            'sideup': ['an apple covered in upwards-facing red arrows saying this side up'],
                            'sisalweave': ['an apple made of woven rope'],
                            'treebark': ['an apple made of tree bark', 'an apple covered in tree bark']},
            'sofa_1': {'blackpepperbrick': ['a sofa made of grey brick', 'a statue of a sofa made of brick'],
                            'bricktextureinternet': ['a sofa made of red brick', 'a statue of a sofa made of red brick'],
                            'chainmail': ['a sofa made of chainmail', 'a sofa wearing chainmail'],
                            'checkerboard': ['a sofa with a red and black checkerboard pattern',
                                             'a sofa textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a sofa covered in colorful dragon scales',
                                            'a sofa with a cover made of dragon scales'],
                            'sideup': ['a sofa covered in upwards-facing red arrows saying this side up'],
                            'sisalweave': ['a sofa made of woven rope'],
                            'treebark': ['a sofa made of tree bark', 'a sofa covered in tree bark']},
        'barrel': {'blackpepperbrick': ["a barrel made of stone brick", "a barrel made out of grey brick",
                                            "a statue of a barrel made of stone bricks"],
                       'bricktextureinternet': ['a barrel made of brick', 'a barrel made of red brick'],
                       'chainmail': ['a barrel made of chainmail', 'a barrel covered in a chainmail mesh'],
                       'checkerboard': ['a barrel made of black and red checkerboard', 'a barrel covered in a checkerboard pattern'],
                       'dragonscale': ['a barrel made of dragon scales', 'a barrel made of colorful dragon scales',
                                       'a barrel covered in dragon scales', 'a barrel covered in colorful dragon scales'],
                       'sideup': ['a barrel covered in this side up stickers', 'a barrel made of upwards-facing red arrows and stickers saying this side up'],
                       'sisalweave': ['a barrel made of rope', 'a barrel made of rope in a sisal weave pattern'],
                       'stainedwood': ['a barrel made of polished hardwood flooring',
                                       'a barrel covered in polished wood panels',
                                       'a barrel made of stained wood panels'],
                       'treebark': ['a barrel made of tree bark', 'a barrel covered in tree bark'],
                       'concrete': ['a stone statue of a barrel', 'a barrel made of concrete', 'a barrel made of stone'],},
            'hippie_shiba': {'blackpepperbrick': ["a statue of a shiba dog made out of grey brick",
                                            "a statue of a shiba dog made of stone bricks"],
                       'bricktextureinternet': ['a statue of a shiba dog made of brick', 'a statue of a shiba dog made of red brick'],
                       'chainmail': ['a statue of a shiba dog made of chainmail', 'a shiba dog covered in a chainmail mesh'],
                       'checkerboard': ['a statue of a shiba dog painted in black and red checkerboard pattern',
                                        'a shiba dog covered in a black and red checkerboard pattern'],
                       'dragonscale': ['a shiba dog with skin made of dragon scales', 'a shiba dog with skin made of colorful dragon scales',
                                       'a statue of a shiba dog covered in colorful dragon scales'],
                       'sideup': ['a shiba dog covered in this side up stickers',
                                  'a statue of a shiba dog made of upwards-facing red arrows and stickers saying this side up'],
                       'sisalweave': ['a statue of a shiba dog made of rope', 'a shiba dog doll made of rope in a sisal weave pattern'],
                       'stainedwood': ['a statue of a shiba dog made of stained wood',
                                       'a statue of a shiba dog covered in stained wood',
                                       ],
                       'treebark': ['a statue of a shiba dog made of tree bark', 'a shiba dog with skin made of tree bark'],
                       'concrete': ['a stone statue of a shiba dog', 'a concrete statue of a shiba dog', 'a shiba dog made of stone']
                       },
            }

jsons = [model + "_slim.json" for model in models]

# First exp is cylinder sds
f = open("./slurm/nopt_pdataprompts", 'w')

for texture in textures:
    for model in models:
        json = model + "_slim.json"
        texturename = texture.split('.')[0]
        captionlist = captions[model][texturename]

        for caption in captionlist:
            expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
            expstr += f" --sdscaption {caption}"
            expstr += f" --expname prompts_{model}_{texturename}_CAPTION_{caption.replace(' ', '_')}"
            f.write(expstr + "\n")
f.close()

In [4]:
### Prompt engineering + controlnet SDS + CSD for parameterization benchmark meshes
# NOTE: try for barrel, shiba
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --sdsloss controlnet csd --spweight sigmoid --arch mlp \
--csd_maxweight 0.5 --lossdistortion confvar --amips --lowmemory --normalizeinit"

textures = ["blackpepperbrick.jpg",
            "bricktextureinternet.jpeg",
            "checkerboard.jpeg", "dragonscale.jpeg", "sideup.png",
             "treebark.jpg"]
models = ['can_soup', 'manzana', 'sofa_1']

params = ['', ', 35 mm', ', 4k', ', award-winning', ', unreal engine']
captions = {'bob_floatie': {'blackpepperbrick': ['a rubber duck made of grey brick', 'a statue of a rubber duck made of brick'],
                            'bricktextureinternet': ['a rubber duck made of red brick', 'a statue of a rubber duck made of red brick'],
                            'chainmail': ['a rubber duck made of chainmail', 'a rubber duck wearing chainmail'],
                            'checkerboard': ['a rubber duck with a red and black checkerboard pattern',
                                             'a rubber duck textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a rubber duck covered in colorful dragon scales',
                                            'a rubber duck with skin made of dragon scales'],
                            'sideup': ['a rubber duck covered in this side up stickers'],
                            'sisalweave': ['a rubber duck made of woven rope'],
                            'treebark': ['a rubber duck made of tree bark', 'a rubber duck covered in tree bark']},
            'can_soup': {'blackpepperbrick': ['a soup can made of grey brick', 'a statue of a soup can made of brick'],
                            'bricktextureinternet': ['a soup can made of red brick', 'a statue of a soup can made of red brick'],
                            'chainmail': ['a soup can made of chainmail', 'a soup can wearing chainmail'],
                            'checkerboard': ['a soup can with a red and black checkerboard pattern',
                                             'a soup can textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a soup can covered in colorful dragon scales',
                                            'a soup can with skin made of dragon scales'],
                            'sideup': ['a soup can covered in this side up stickers'],
                            'sisalweave': ['a soup can made of woven rope'],
                            'treebark': ['a soup can made of tree bark', 'a soup can covered in tree bark']},
            'laptop_for_music_production': {'blackpepperbrick': ['a laptop made of grey brick', 'a statue of a laptop made of brick'],
                            'bricktextureinternet': ['a laptop made of red brick', 'a statue of a laptop made of red brick'],
                            'chainmail': ['a laptop made of chainmail', 'a laptop wearing chainmail'],
                            'checkerboard': ['a laptop with a red and black checkerboard pattern',
                                             'a laptop textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a laptop covered in colorful dragon scales',
                                            'a laptop in a cover made of dragon scales'],
                            'sideup': ['a laptop covered in this side up stickers'],
                            'sisalweave': ['a laptop made of woven rope'],
                            'treebark': ['a laptop made of tree bark', 'a laptop covered in tree bark']},
            'manzana': {'blackpepperbrick': ['an apple made of grey brick', 'a statue of an apple made of brick'],
                            'bricktextureinternet': ['an apple made of red brick', 'a statue of an apple made of red brick'],
                            'chainmail': ['an apple made of chainmail', 'an apple wearing chainmail'],
                            'checkerboard': ['an apple with a red and black checkerboard pattern',
                                             'an apple textured with a red and black checkerboard pattern'],
                            'dragonscale': ['an apple covered in colorful dragon scales',
                                            'an apple with skin made of dragon scales'],
                            'sideup': ['an apple covered in this side up stickers'],
                            'sisalweave': ['an apple made of woven rope'],
                            'treebark': ['an apple made of tree bark', 'an apple covered in tree bark']},
            'sofa_1': {'blackpepperbrick': ['a sofa made of grey brick', 'a statue of a sofa made of brick'],
                            'bricktextureinternet': ['a sofa made of red brick', 'a statue of a sofa made of red brick'],
                            'chainmail': ['a sofa made of chainmail', 'a sofa wearing chainmail'],
                            'checkerboard': ['a sofa with a red and black checkerboard pattern',
                                             'a sofa textured with a red and black checkerboard pattern'],
                            'dragonscale': ['a sofa covered in colorful dragon scales',
                                            'a sofa with a cover made of dragon scales'],
                            'sideup': ['a sofa covered in this side up stickers'],
                            'sisalweave': ['a sofa made of woven rope'],
                            'treebark': ['a sofa made of tree bark', 'a sofa covered in tree bark']},
        'barrel': {'blackpepperbrick': ["a barrel made of stone brick", "a barrel made out of grey brick",
                                            "a statue of a barrel made of stone bricks"],
                       'bricktextureinternet': ['a barrel made of brick', 'a barrel made of red brick'],
                       'chainmail': ['a barrel made of chainmail', 'a barrel covered in a chainmail mesh'],
                       'checkerboard': ['a barrel made of black and red checkerboard', 'a barrel covered in a checkerboard pattern'],
                       'dragonscale': ['a barrel made of dragon scales', 'a barrel made of colorful dragon scales',
                                       'a barrel covered in dragon scales', 'a barrel covered in colorful dragon scales'],
                       'sideup': ['a barrel covered in this side up stickers'],
                       'sisalweave': ['a barrel made of rope', 'a barrel made of rope in a sisal weave pattern'],
                       'stainedwood': ['a barrel made of polished hardwood flooring',
                                       'a barrel covered in polished wood panels',
                                       'a barrel made of stained wood panels'],
                       'treebark': ['a barrel made of tree bark', 'a barrel covered in tree bark'],
                       'concrete': ['a stone statue of a barrel', 'a barrel made of concrete', 'a barrel made of stone'],},
            'hippie_shiba': {'blackpepperbrick': ["a statue of a shiba dog made out of grey brick",
                                            "a statue of a shiba dog made of stone bricks"],
                       'bricktextureinternet': ['a statue of a shiba dog made of brick', 'a statue of a shiba dog made of red brick'],
                       'chainmail': ['a statue of a shiba dog made of chainmail', 'a shiba dog covered in a chainmail mesh'],
                       'checkerboard': ['a statue of a shiba dog painted in black and red checkerboard pattern',
                                        'a shiba dog covered in a black and red checkerboard pattern'],
                       'dragonscale': ['a shiba dog with skin made of dragon scales', 'a shiba dog with skin made of colorful dragon scales',
                                       'a statue of a shiba dog covered in colorful dragon scales'],
                       'sideup': ['a shiba dog covered in this side up stickers',
                                  'a statue of a shiba dog made of upwards-facing red arrows and stickers saying this side up'],
                       'sisalweave': ['a statue of a shiba dog made of rope', 'a shiba dog doll made of rope in a sisal weave pattern'],
                       'stainedwood': ['a statue of a shiba dog made of stained wood',
                                       'a statue of a shiba dog covered in stained wood',
                                       ],
                       'treebark': ['a statue of a shiba dog made of tree bark', 'a shiba dog with skin made of tree bark'],
                       'concrete': ['a stone statue of a shiba dog', 'a concrete statue of a shiba dog', 'a shiba dog made of stone']
                       },
            }

# with open("./data/textureorientation/pdataall.txt", 'r') as f:
#     models = f.read().splitlines()
jsons = [model + "_slim.json" for model in models]

# First exp is cylinder sds
f = open("./slurm/nopt_peng", 'w')

for param in params:
    for texture in textures:
        for model in models:
            json = model + "_slim.json"
            texturename = texture.split('.')[0]
            captionlist = captions[model][texturename]

            for caption in captionlist:
                caption = caption + param
                expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                expstr += f" --sdscaption {caption}"
                expstr += f" --expname peng_{model}_{texturename}_CAPTION_{caption.replace(' ', '_')}"
                f.write(expstr + "\n")
f.close()

In [1]:
### Controlnet
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 --sdsloss controlnet \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.5 --lossdistortion confvar --amips \
--sdscheckpointing --clipcheckpointing"

textures = ['bricktextureinternet.jpeg', 'cactus.jpeg', 'checkerboardrot.png']
jsons = ['coarsecut_rot.json', 'coarsecut_tutte.json',
    'monkey_rot.json', 'monkey_tutte.json',
         'torus_rot.json', 'torus_tutte.json',
         'mustache_tutte.json', 'mustache_slim.json', '40988_rot.json',
         '40988_tutte.json']

# First exp is cylinder sds
f = open("./slurm/nopt_controlnet", 'w')

for texture in textures:
    for json in jsons:
        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
        name = json.replace('.json', '')
        texturename = texture.split('.')[0]
        expstr += f" --expname controlnet_{name}_{texturename}"
        f.write(expstr + "\n")

f.close()

In [4]:
### Conformal only baseline
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation --sdsloss csd \
--resolution 256 --spweight sigmoid --arch mlp --lossdistortion confvar --amips --showtexture --sdscheckpointing --clipcheckpointing"

textures = ['bricktextureinternet', 'checkerboard', 'fishscale']
jsons = ['coarsecut_rot.json', 'coarsecut_tutte.json']

# First exp is cylinder sds
f = open("./slurm/nopt_confbase", 'w')

for texture in textures:
    for json in jsons:
        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture}.jpeg --data_file {json} --test_file {json}"
        name = json.replace('.json', '')

        expstr += f" --expname confbase_{name}_{texture}"
        f.write(expstr + "\n")

# Checkerboard only
textures = ['checkerboard.jpeg', 'checkerboardrot.png', 'checkerboardproj.png']
jsons = ['coarsecut_rot.json', 'coarsecut_tutte.json',
         'torus_tutte.json',
         '40988_rot.json', '40988_tutte.json',
         'monkey_rot.json', 'monkey_tutte.json',]

for texture in textures:
    for json in jsons:
        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
        name = json.replace('.json', '')
        texturename = texture.split('.')[0]

        expstr += f" --expname confbase_{name}_{texturename}"
        f.write(expstr + "\n")

f.close()

In [1]:
### UV log loss
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation --sdsloss csd \
--visualloss sds --resolution 256 --spweight sigmoid --arch mlp --lossdistortion confvar --amips --showtexture"

# First exp is cylinder sds
f = open("./slurm/nopt_logloss", 'w')

# Checkerboard only
textures = ['checkerboard.jpeg', 'checkerboardrot.png', 'checkerboardproj.png']
jsons = ['coarsecut_rot.json', 'coarsecut_tutte.json',
         'torus_tutte.json',
         'monkey_rot.json', 'monkey_tutte.json',]

for texture in textures:
    for json in jsons:
        expstr = basestr + f" --uvlogloss --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
        name = json.replace('.json', '')
        texturename = texture.split('.')[0]

        expstr += f" --expname logloss_{name}_{texturename}"
        f.write(expstr + "\n")

f.close()

In [2]:
### HUGE RUN
import os

basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 --sdsloss csd \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.5 --lossdistortion confvar --amips"

with open('./data/textureorientation/alltextures.txt', 'r') as f:
    alltextures = [line.strip() for line in f.readlines()]

alljsons = []
for jsonfile in os.listdir('./data/textureorientation'):
    if jsonfile.endswith('.json'):
        alljsons.append(jsonfile)

# We can only batch array jobs from 1 - 999

count = 0
fcount = 0
f = open(f"./slurm/nopt_all_{fcount}", 'w')

for json in alljsons:
    for texture in alltextures:
        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
        name = json.replace('.json', '')
        texturename = texture.split('.')[0]
        expstr += f" --expname all_{name}_{texturename}"
        f.write(expstr + "\n")
        count += 1

        if count >= 100:
            f.close()
            count = 0
            fcount += 1
            f = open(f"./slurm/nopt_all_{fcount}", 'w')
f.close()

In [3]:
### Testing SDS on neural opt texture orientation
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 --sdsloss csd \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.5 --lossdistortion confvar --amips \
--sdscheckpointing --clipcheckpointing"

# First exp is cylinder sds
f = open("./slurm/nopt_sds", 'w')

# Checkerboard only
textures = ['blackpepperbrick.jpg', 'checkerboard.jpeg', 'checkerboardrot.png', 'checkerboardproj.png',
            'corduroy.jpg', 'marble.jpg', 'plant.jpg']
jsons = ['coarsecut_rot.json', 'coarsecut_rotshrink.json', 'coarsecut_tutte.json',
         'torus_rot.json', 'torus_rotshrink.json', 'torus_tutte.json',
         'monkey_rot.json', 'monkey_rotshrink.json', 'monkey_tutte.json',
         '40988_rot.json', '40988_tutte.json',]
for texture in textures:
    for json in jsons:
        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
        name = json.replace('.json', '')
        texturename = texture.split('.')[0]
        expstr += f" --expname standard_{name}_{texturename}"
        f.write(expstr + "\n")

f.close()

In [1]:
### Check CSD + AMIPS hyperparams to look for what works over all different initializations
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.2"

textures = ['checkerboard.jpeg', 'checkerboardrot.png']
jsons = ['coarsecut_rot.json', 'coarsecut_rotshrink.json', 'coarsecut_tutte.json']
max_t = [0.98, 0.5, 0.7]

# TODO: Choose correct LR + conformal distortion energy!!!
f = open("./slurm/nopt_hyperparam", 'w')

    for t in max_t:
        for texture in textures:
            for json in jsons:
                expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                name = json.replace('.json', '')
                texturename = texture.split(".")[0]

                expstr += f" --sds_max_step_percent {t}"
                expname = f"{name}_{texturename}_{t}_lr{lr}"

                expstr += f" --sdsloss csd"
                expname += "_csd"

                expstr += f" --lossdistortion mips"
                expname += "_mips"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
f.close()

In [3]:
### Check CSD + AMIPS hyperparams to look for what works over all different initializations
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.2"

textures = ['checkerboard.jpeg', 'checkerboardrot.png', 'checkerboardproj.png']
jsons = ['coarsecut_rot.json']
distortions = ['mips', 'confvar', 'confvar_amips']
dweights = [1, 10]
csd_weights = [0.2, 0.5]

f = open("./slurm/nopt_distortion", 'w')

for texture in textures:
    for dweight in dweights:
        for csd in csd_weights:
            for distortion in distortions:
                    for json in jsons:
                        expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                        name = json.replace('.json', '')
                        texturename = texture.split(".")[0]

                        expname = f"{name}_{texturename}_{distortion}_d{dweight}_csd{csd}"

                        expstr += f" --sdsloss csd --lr 1e-4 --distortion_weight {dweight} --csd_maxweight {csd}"

                        if "amips" in distortion:
                            expstr += f" --lossdistortion confvar --amips"
                        else:
                            expstr += f" --lossdistortion {distortion}"

                        expstr += f" --expname {expname}"
                        f.write(expstr + "\n")
f.close()

In [1]:
### Investigate how different settings influence the good checkerboard convergence
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.2 --arapnorm"

textures = ['checkerboard.jpeg', 'checkerboardrot.png']
jsons = ['coarsecut_rot.json']
max_t = [0.5, 0.2]

f = open("./slurm/nopt_check", 'w')

for distortion in [False, True]:
    for csd in [False, True]:
        for t in max_t:
            for texture in textures:
                for json in jsons:
                    expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
                    name = json.replace('.json', '')
                    texturename = texture.split(".")[0]

                    expstr += f" --sds_max_step_percent {t}"
                    expname = f"{name}_{texturename}_{t}"

                    if csd:
                        expstr += f" --sdsloss csd"
                        expname += "_csd"
                    else:
                        expstr += f" --sdsloss text2img"

                    if distortion:
                        expstr += f" --lossdistortion mips"
                        expname += "_mips"

                    expstr += f" --expname {expname}"
                    f.write(expstr + "\n")
f.close()

In [2]:
### Testing SDS on neural opt texture orientation
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --csd_maxweight 0.2 --arapnorm"

textures = ['checkerboard']
jsons = ['coarsecut_rot.json', 'coarsecut_rotshrink.json', 'coarsecut_tutte.json']

# First exp is cylinder sds (compare csd with no csd)
f = open("./slurm/nopt_cylchecker", 'w')

for distortion in [False, True]:
    for csd in [False, True]:
        for texture in textures:
            for json in jsons:
                expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture}.jpeg --data_file {json} --test_file {json}"
                name = json.replace('.json', '')
                expname = f"{name}_{texture}"

                if csd:
                    expstr += f" --sdsloss csd"
                    expname += "_csd"
                else:
                    expstr += f" --sdsloss text2img"

                if distortion:
                    expstr += f" --lossdistortion mips"
                    expname += "_mips"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
f.close()

# Checkerboard only
# textures = ['checkerboard.jpeg', 'checkerboardrot.png']
# jsons = ['coarsecut_rot.json', 'coarsecut_rotshrink.json', 'coarsecut_tutte.json',
#          'torus_rot.json', 'torus_rotshrink.json', 'torus_tutte.json',
#          'monkey_rot.json', 'monkey_rotshrink.json', 'monkey_tutte.json',]

# for csd in [False, True]:
#     for texture in textures:
#         for json in jsons:
#             expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
#             name = json.replace('.json', '')
#             texturename = texture.split('.')[0]

#             if csd:
#                 expstr += f" --sdsloss csd"
#                 expstr += f" --expname {name}_{texture}_csd"
#             else:
#                 expstr += f" --sdsloss text2img"
#                 expstr += f" --expname {name}_{texture}"

#             f.write(expstr + "\n")

# f.close()

In [1]:
### Testing SDS on neural opt texture orientation
basestr = "python ./training_scripts/train.py --nviews 5 --undopin --init precut \
--rasterizer kaolin --noencoder --experiment_type DEFAULT --size_train 1 --size_test 1 \
--epochs 5000 --val_interval 10 --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 \
--projectname texture --lr 1e-4 --targets_per_batch 16 --workers 0 --valrenderratio 0.5 \
--accumulate_grad_batches 1 --outputdir ./outputs/neuralopt/texture --identity \
--vertexdim 32 --facedim 32 --cuteps 1 --minweight 1e-7 --top_k_eig 15 \
--root_dir_train ./data/textureorientation --root_dir_test ./data/textureorientation \
--resolution 256 --visualloss sds --spweight sigmoid --arch mlp --sdsloss csd"

# First exp is cylinder sds
f = open("./slurm/nopt_sds", 'w')

# Checkerboard only
textures = ['blackpepperbrick.jpg', 'checkerboard.jpeg', 'checkerboardrot.png', 'checkerboardproj.png',
            'corduroy.jpg', 'marble.jpg', 'plant.jpg']
jsons = ['coarsecut_rot.json', 'coarsecut_rotshrink.json', 'coarsecut_tutte.json',
         'torus_rot.json', 'torus_rotshrink.json', 'torus_tutte.json',
         'monkey_rot.json', 'monkey_rotshrink.json', 'monkey_tutte.json',
         '40988_rot.json', '40988_tutte.json',]

for rangeloss in [False, True]:
    for texture in textures:
        for json in jsons:
            expstr = basestr + f" --textureimg ./data/textureorientation/textures/{texture} --data_file {json} --test_file {json}"
            name = json.replace('.json', '')
            texturename = texture.split('.')[0]

            if rangeloss:
                expstr += f" --uvrangeloss"
                expstr += f" --expname {name}_{texturename}_uvrange_csd"
            else:
                expstr += f" --expname {name}_{texturename}_csd"
            f.write(expstr + "\n")

f.close()

In [1]:
### Baseline NJF with GT J supervision
#### Coseg chairs, vases, dawand
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--shuffle_triangles",
              "--projectname", "multishape", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "0.5",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/baseline",
              "--identity", "--vertexdim", "32", "--facedim", "32",
              "--hardpoisson", "jacobians", "--cuteps", "1", "--minweight", "1e-7", "--top_k_eig", "15",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/baseline"

f = open("./slurm/baseline_learn", 'w')
dirpairs = [['coseg_chairs_train', 'coseg_chairs_test'],['dawand_train', 'dawand_test']]
losstype = ['gtjacobianloss']

for traindir, testdir, in dirpairs:
    for loss in losstype:
        expstr = basestr + f" --root_dir_train ./data/{traindir} --data_file {traindir}.json"
        expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
        expstr += f" --{loss} --spweight sigmoid"
        expstr += f" --arch mlp"

        expname = f"{traindir}_mlp_{loss}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")

f.close()

In [1]:
#### Multi-shape: chairs, vases, dawand
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson",
              "--projectname", "multishape", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--vertexdim", "32", "--facedim", "32",
              "--hardpoisson", "uv", "--cuteps", "0.1", "--minweight", "1e-7", "--top_k_eig", "15",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/multishape_pointnet", 'w')
dirpairs = [['coseg_chairs_train', 'coseg_chairs_test'], ['coseg_vases_train', 'coseg_vases_test']]
losstype = ['gtnetworkloss']
arches = ['diffusionnet', 'fullconv2']
spweights = ['sigmoid', 'nonzero']
inits = [None, 'precut']

for traindir, testdir, in dirpairs:
    for arch in arches:
        for loss in losstype:
            for init in inits:
                expstr = basestr + f" --root_dir_train ./data/{traindir} --data_file {traindir}.json"
                expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
                expstr += f" --{loss} --spweight sigmoid"
                expstr += f" --softpoisson edges --arch {arch}"

                if init:
                    expstr += f" --init {init} --initweightinput --initjinput"

                expname = f"{traindir}_{loss}_arch{arch}_init{init}"
                expstr += f" --expname {expname}"
                f.write(expstr + "\n")

for traindir, testdir, in dirpairs:
    for arch in arches:
        for init in inits:
            expstr = basestr + f" --root_dir_train ./data/{traindir} --data_file {traindir}.json"
            expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
            expstr += " --lossdistortion arap --arapnorm --stitchingloss edgecutloss --spweight sigmoid --seamlessedgecut"
            expstr += f" --softpoisson edges --arch {arch}"

            if init:
                expstr += f" --init {init} --initweightinput --initjinput"

            expname = f"{traindir}_unsupervised_arch{arch}_init{init}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")

f.close()

In [3]:
#### Dataset learning
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "multishape", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "32", "--facedim", "32",
              "--hardpoisson", "uv", "--cuteps", "0.1", "--minweight", "1e-7"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/syntheticdataset", 'w')
dirpairs = [['dawand_train', 'dawand_test'],]
losstype = ['gtuvloss', 'gtnetworkloss']
arches = ['diffusionnet', 'fullconv']

for traindir, testdir, in dirpairs:
    for arch in arches:
        for loss in losstype:
            expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
            expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
            expstr += f" --{loss}"
            expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid"

            expname = f"{traindir}_{loss}_arch{arch}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")

spweights = ['sigmoid', 'nonzero']
losstype = ['gtuvloss', 'gtnetworkloss']
arches = ['fullconv2']
for traindir, testdir, in dirpairs:
    for spweight in spweights:
        for arch in arches:
            for loss in losstype:
                expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
                expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
                expstr += f" --{loss} --spweight {spweight}"
                expstr += f" --softpoisson edges --arch {arch}"

                expname = f"{traindir}_{loss}_arch{arch}_spweight{spweight}"
                expstr += f" --expname {expname}"
                f.write(expstr + "\n")

spweights = ['sigmoid', 'nonzero']
arches = ['fullconv2', 'diffusionnet']
dirpairs = [['dawand_train_gtinit', 'dawand_test_gtinit']]
for traindir, testdir, in dirpairs:
    for spweight in spweights:
        for arch in arches:
            expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
            expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
            expstr += f" --stitchingloss edgecutloss --lossdistortion arap --arapnorm --spweight {spweight}"
            expstr += f" --softpoisson edges --arch {arch}"

            expname = f"{traindir}_unsup_arch{arch}_spweight{spweight}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [5]:
#### Single shape learning
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "singleshape_oc", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "32", "--facedim", "32",
              "--hardpoisson", "uv", "--cuteps", "0.1", "--minweight", "1e-7"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/singleshape_oc_v2", 'w')
dirpairs = [['optcuts_bunny', 'optcuts_bunny_test'], ['optcuts_sample0', 'optcuts_sample0_test'],
            ['optcuts_sample1', 'optcuts_sample1_test'],]
losstype = ['gtuvloss', 'gtnetworkloss']
spweights = ['sigmoid', 'nonzero']
arches = ['diffusionnet', 'fullconv2']

for traindir, testdir, in dirpairs:
    for arch in arches:
        for loss in losstype:
            for spweight in spweights:
                expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
                expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
                expstr += f" --{loss}"

                expstr += f" --softpoisson edges --arch {arch} --spweight {spweight}"

                expname = f"{traindir}_{loss}_arch{arch}_spweight{spweight}"
                expstr += f" --expname {expname}"
                f.write(expstr + "\n")

# dirpairs = [['optcuts_bunny', 'optcuts_bunny_test', 'bunny'], ['optcuts_sample0', 'optcuts_sample0_test', 'sample0'],
#             ['optcuts_sample1', 'optcuts_sample1_test', 'sample1'],]
# for traindir, testdir, solodir in dirpairs:
#     for arch in arches:
#         for loss in ['weightededgecutloss']:
#             expstr = basestr + f" --init isometric --ninit 1 --root_dir_train ./data/{traindir} --data_file {solodir}.json"
#             expstr += f" --root_dir_test ./data/{testdir} --test_file {solodir}.json"
#             if loss == "weightededgecutloss":
#                 expstr += f" --stitchingloss weightededgecutloss --lossdistortion arap --arapnorm"
#             else:
#                 expstr += f" --{loss}"

#             expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid"

#             expname = f"{traindir}_{loss}_arch{arch}"
#             expstr += f" --expname {expname}"
#             f.write(expstr + "\n")

f.close()

In [1]:
#### Weighted edge stitching
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "singleshape_unsupervised", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "32", "--facedim", "32",
              "--hardpoisson", "uv", "--cuteps", "0.1", "--minweight", "1e-7"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt"

f = open("./slurm/weightedstitching", 'w')
dirs = ['bunny_furthestcut', 'bunny_optcuts', 'sample0_furthestcut', 'sample0_optcuts']
spweights = ['sigmoid', 'nonzero']
arches = ['diffusionnet', 'fullconv2']

for traindir in dirs:
    for arch in arches:
        for spweight in spweights:
            expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/frequencyweighttest --data_file {traindir}.json"
            expstr += f" --root_dir_test ./data/frequencyweighttest --test_file {traindir}.json"
            expstr += f" --stitchingloss weightededgecutloss --lossdistortion arap --arapnorm"
            expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid --spweight {spweight}"

            expname = f"{traindir}_{spweight}_arch{arch}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")

f.close()

In [2]:
#### GT Vase neural optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "5000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "nopt", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt",
              "--identity", "--vertexdim", "32", "--facedim", "32"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/ocinit_nopt", 'w')
traindir = "coseg_vases"
testdir = "coseg_vases"
arches = ['diffusionnet', 'fullconv']

for arch in arches:
    expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file vase_nopt.json"
    expstr += f" --root_dir_test ./data/{testdir} --test_file vase_nopt.json"
    expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid"
    expstr += f" --lossdistortion arap --stitchingloss edgecutloss --seamlessedgecut --seamlessdelta 0.005"

    if arch == "fullconv":
        expstr += f" --pool_res 700 600 500"

    expname = f"ocinit_nopt_arch{arch}"
    expstr += f" --expname {expname}"
    f.write(expstr + "\n")
f.close()

In [ ]:
#### Cone single shape learning
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "1000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "32", "--facedim", "32"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/cone_lr", 'w')
traindir = "cone_lr"
testdir = "cone_lr_test"
losstype = ['gtuvloss', 'gtnetworkloss']
inittopos = [True, False]
arches = ['diffusionnet', 'fullconv']

for arch in arches:
    for loss in losstype:
        expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
        expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
        expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid --{loss}"

        expname = f"{traindir}_{loss}_arch{arch}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [1]:
#### Cone LR learning arch experiments
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "32", "--facedim", "32"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/cone_lr", 'w')
traindir = "cone_lr"
testdir = "cone_lr_test"
losstype = ['gtuvloss', 'gtnetworkloss']
inittopos = [True, False]
arches = ['diffusionnet', 'fullconv']

for arch in arches:
    for loss in losstype:
        expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
        expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
        expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid --{loss}"

        if arch == "fullconv":
            expstr += f" --pool_res 700 600 500"

        expname = f"{traindir}_{loss}_arch{arch}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [2]:
#### Fullconv cone unsupervised learning
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "30000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
datas = ['cone_lr']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/sp_cone_unsupervised", 'w')
for data in datas:
    expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{data} --data_file {data}.json --root_dir_test ./data/{data}_test --test_file {data}_test.json"
    expstr += f" --softpoisson edges --arch fullconv --pool_res 700 600 500"
    expstr += f" --arapnorm --edgecut_weight 0.01"
    expstr += f" --spweight sigmoid"
    expname = f"usp_{data}"
    expstr += f" --expname {expname}"
    f.write(expstr + "\n")
f.close()

In [1]:
#### Simple cone learning w/ new dnet architecture
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "64",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/cone_simple_fullconv", 'w')
traindir = "optcuts_cone_simple"
testdir = "optcuts_cone_simple_test"
losstype = ['gtuvloss', 'gtnetworkloss']
inittopos = [True, False]

for inittopo in inittopos:
    for loss in losstype:
        expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
        expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
        expstr += f" --softpoisson edges --arch fullconv --spweight sigmoid --{loss}"

        if inittopo:
            expstr += f" --inittopo"

        expname = f"{traindir}_{loss}_inittopo{inittopo}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [2]:
#### Simple cone learning w/ new dnet architecture
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--identity", "--vertexdim", "64",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/cone_simple", 'w')
traindir = "optcuts_cone_simple"
testdir = "optcuts_cone_simple_test"
losstype = ['gtuvloss', 'gtnetworkloss']
inittopos = [True, False]

for inittopo in inittopos:
    for loss in losstype:
        expstr = basestr + f" --init precut --ninit 1 --root_dir_train ./data/{traindir} --data_file {traindir}.json"
        expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
        expstr += f" --softpoisson edges --arch diffusionnet --spweight sigmoid --{loss}"

        if inittopo:
            expstr += f" --inittopo"

        expname = f"{traindir}_{loss}_inittopo{inittopo}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [2]:
#### Direct UV Learning
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/directuv",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--projectname", "directuv_nopt", "--cuteps", "5e-2",
              "--facedim", "32", "--vertexdim", "32"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/directuv/"
datadirs = ['optcuts_cone_simple']
testdirs = ["optcuts_cone_simple_test"]

f = open("./slurm/directuv_cone", 'w')
for data in datadirs:
    for testdir in testdirs:
        expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{testdir} --data_file {data}.json --test_file {testdir}.json"
        expstr += f" --arch diffusionnet --init precut --directuv --gtuvloss"

        expname = f"directuv_{data}_test{testdir}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [1]:
#### Manual vase learning
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "gtuv", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--vertexdim", "32", "--facedim", "32",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"

f = open("./slurm/vases", 'w')
dirs = [("vases_manual_train", "vases_manual_test")]
losstype = ['gtuvloss', 'gtnetworkloss']
extrinsics = [True, False]
inits = [True]
arches = ['diffusionnet', 'fullconv']

for traindir, testdir in dirs:
    for loss in losstype:
        for extrinsic in extrinsics:
            for arch in arches:
                for init in inits:
                    expstr = basestr + f" --root_dir_train ./data/{traindir} --data_file {traindir}.json"
                    expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json --hks"
                    expstr += f" --softpoisson edges --arch {arch} --spweight sigmoid --{loss}"

                    if init:
                        expstr += f" --init precut --ninit 1"

                    if extrinsic:
                        expstr += " --extrinsic"

                    expname = f"{arch}_vasemanual_init{init}_{loss}_extrinsic{extrinsic}"
                    expstr += f" --expname {expname}"
                    f.write(expstr + "\n")
f.close()

In [3]:
#### Direct UV Nopt
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/directuv",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--projectname", "directuv_nopt", "--cuteps", "5e-2",
              "--facedim", "16", "--vertexdim", "16"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', 'tetrahedron_sub1', "cylinder_nocut"]
inits = ['isometric', 'tutte']
edgeweights = [0, 0.1]
stitchschedule = [None, 'constant']
normallosses = [False, True]

f = open("./slurm/directuv_nopt", 'w')
for data in datadirs:
    for init in inits:
        for eweight in edgeweights:
            for schedule in stitchschedule:
                for normal in normallosses:
                    if eweight == 0 and schedule is not None:
                        continue

                    if eweight == 0 and init == 'isometric':
                        continue

                    if schedule is not None and init == "isometric":
                        continue

                    if normal and data not in ['cone', 'cylinder_nocut', 'tetrahedron_sub1']:
                        continue

                    expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json --test_file {data}.json"
                    expstr += f" --arch diffusionnet --init {init} --directuv"
                    expstr += " --lossdistortion arap --arapnorm --stitchingloss edgecutloss"
                    expstr += f" --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0"

                    if normal:
                        expstr += " --normalloss"

                    if schedule is not None:
                        expstr += f" --stitchschedule {schedule}"

                    if schedule == 'constant':
                        expstr += f" --stitchschedule_constant 0.5"

                    if 'tetrahedron_sub1' in data:
                        expstr += f" --top_k_eig 8"
                    elif 'tetrahedron4' in data:
                        expstr += f" --top_k_eig 3"

                    expname = f"{data}_init{init}_{eweight}_{schedule}_nl{normal}"
                    expstr += f" --expname {expname}"
                    f.write(expstr + "\n")
f.close()

In [1]:
#### SP SDS
import numpy as np
import os

basestr = ["--init", "slim", "--edgeweightonly", "--viziter", "20",
            "--texturetext", "a cylinder with a red and black checkerboard pattern"]

basestr = "python ./opt_sds.py " + " ".join(basestr)
baseoutput = "./outputs/optimization"
datadirs = ['cylinder']
lrs = [1e-3]

textures = ["./data/textures/discretemap.png", "./data/textures/discreterainbow.jpg",
            "./data/textures/speckledgradient.png", "./data/textures/checkerboard.jpeg"]
highres = [False, True]

f = open("./slurm/sp_sds", 'w')
for data in datadirs:
    for texture in textures:
        for lr in lrs:
            for res in highres:
                texturename = os.path.basename(texture).split(".")[0]
                expstr = basestr + f" --objdir ./data/{data}/{data}.obj --lr {lr} --imageloss ./data/textures/gt/{data}_{texturename}_gt"
                expstr += f" --textureimg {texture} --savedir ./outputs/optimization --distortionloss --edgecutloss --seamlessedgecut"
                expstr += f" --no_weight_optimization"

                if res:
                    expstr += " --renderresolution 512"
                else:
                    expstr += " --renderresolution 128"

                expname = f"spsds_{data}_{texturename}_lr{lr}_hr{res}"
                expstr += f" --savename {expname} --imagelossn 5 --niters 1000"
                f.write(expstr + "\n")
f.close()

In [1]:
#### Single triangle SDS
import numpy as np
import os

basestr = ["--init", "slim", "--edgeweightonly", "--viziter", "20",
            "--texturetext", "a cylinder with a red and black checkerboard pattern"]


basestr = "python ./opt_sds_simple.py " + " ".join(basestr)
baseoutput = "./outputs/optimization"
datadirs = ['cylinder']
lrs = [1e-2]
initthetas = [0.0, np.pi/4.0, np.pi/2.0]

textures = ["./data/textures/discretemap.png", "./data/textures/discreterainbow.jpg",
            "./data/textures/speckledgradient.png", "./data/textures/checkerboard.jpeg"]

f = open("./slurm/simple_sds", 'w')
for data in datadirs:
    for texture in textures:
        for lr in lrs:
            for theta in initthetas:
                texturename = os.path.basename(texture).split(".")[0]
                expstr = basestr + f" --objdir ./data/{data}/{data}.obj --lr {lr} --inittheta {theta} --imageloss ./outputs/{data}_{texturename}_gt"
                expstr += f" --textureimg {texture}"
                expname = f"sds_{data}_{texturename}_lr{lr}_theta{theta:04f}"
                expstr += f" --savename {expname} --imagelossn 5 --niters 1000"
                f.write(expstr + "\n")
f.close()

In [4]:
## OptCuts GT: generate jsons
import os, sys, json, argparse, subprocess, glob, shutil, re
traindir = "./data/optcuts_train"
testdirgeo = "./data/optcuts_test_geometry"
testdirsample = "./data/optcuts_test_sample"

# Generate training json
train_json = {'pairs': []}

for meshfolder in os.listdir(traindir):
    if not os.path.exists(os.path.join(traindir, meshfolder, "gt_j.pt")):
        continue
    train_json['pairs'].append([f"{meshfolder}/{meshfolder}.obj", f"{meshfolder}/{meshfolder}.obj"])

with open(os.path.join(traindir, f"optcuts_train.json"), "w") as f:
    json.dump(train_json, f)

test_geo_json = {'pairs': []}
for meshfolder in os.listdir(testdirgeo):
    if not os.path.exists(os.path.join(testdirgeo, meshfolder, "gt_j.pt")):
        continue
    test_geo_json['pairs'].append([f"{meshfolder}/{meshfolder}.obj", f"{meshfolder}/{meshfolder}.obj"])

with open(os.path.join(testdirgeo, f"optcuts_test_geometry.json"), "w") as f:
    json.dump(test_geo_json, f)

test_sample_json = {'pairs': []}
for meshfolder in os.listdir(testdirsample):
    if not os.path.exists(os.path.join(testdirsample, meshfolder, "gt_j.pt")):
        continue
    test_sample_json['pairs'].append([f"{meshfolder}/{meshfolder}.obj", f"{meshfolder}/{meshfolder}.obj"])

with open(os.path.join(testdirsample, f"optcuts_test_sample.json"), "w") as f:
    json.dump(test_sample_json, f)

In [1]:
#### Learning experiments with optcuts gt
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--facedim", "128",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005", "--edgecut_weight", "1",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"
traindir = "chairs_train_cut"
testdir = "chairs_test_cut"
losstype = ['gtuvloss', 'gtnetworkloss']

# TODO: precut inits
inits = [None]

f = open("./slurm/oc_chair", 'w')
for loss in losstype:
    for init in inits:
        expstr = basestr + f" --root_dir_train ./data/{traindir} --data_file {traindir}.json"
        expstr += f" --root_dir_test ./data/{testdir} --test_file {testdir}.json"
        expstr += f" --softpoisson edges --arch diffusionnet --spweight sigmoid --{loss}"

        if init:
            expstr += f" --init {init} --initweightinput --initjinput"

        expname = f"{testdir}_{loss}_init{init}"
        expstr += f" --expname {expname}"
        f.write(expstr + "\n")

f.close()

In [2]:
#### Diffusionnet cut learning (larger dataset)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "30000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
softmaxs = [False, True]
datas = ['dataset1']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/cutlearn_dataset1", 'w')
for data in datas:
    for softmax in softmaxs:
        for removecut in [False]:
            expstr = basestr + f" --init tutte --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
            expstr += f" --min_cuts 4 --max_cuts 18 --simplecut --arapnorm --edgecut_weight 0.01"
            expstr += f" --accumulate_grad_batches 5"

            if softmax:
                expstr += f" --spweight softmax"
            else:
                expstr += f" --spweight sigmoid"

            expname = f"{data}_softmax{softmax}"

            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [7]:
#### Diffusionnet cut learning (greedycuts init)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "30000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
softmaxs = [False, True]
datas = ['greedycut_dataset']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/greedycuts_learn", 'w')
for data in datas:
    for softmax in softmaxs:
        for removecut in [False]:
            expstr = basestr + f" --init tutte --ninit 1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
            expstr += f" --arapnorm --edgecut_weight 0.01"
            expstr += f" --accumulate_grad_batches 3"

            if softmax:
                expstr += f" --spweight softmax"
            else:
                expstr += f" --spweight nonzero"

            expname = f"{data}_softmax{softmax}"

            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [2]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone']
distortionloss = [True, False]
sparseloss = [True, False]
edgeloss = [True, False]
inits = ['isometric', 'tutte']

f = open("./slurm/cone_sds", 'w')
for data in datadirs:
    for distortion in distortionloss:
        for sparse in sparseloss:
            for edge in edgeloss:
                for init in inits:
                    expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                    expstr += " --sdsloss --textureimg ./data/textures/cactus.jpeg --texturetext a cone with cactus skin"
                    expstr += f" --softpoisson edges --arch diffusionnet --init {init}"

                    if sparse:
                        expstr += " --sparsecutsloss --sparsecuts_weight 0.1"
                    if distortion:
                        expstr += " --lossdistortion arap --arapnorm"
                    if edge:
                        expstr += f" --stitchingloss edgecutloss --edgecut_weight 0.05 --edgecut_weight_min 0 --ninit 1"

                    expname = f"{data}_init{init}_dist{distortion}_sparse{sparse}_edge{edge}"
                    expstr += f" --expname {expname}"
                    f.write(expstr + "\n")
    f.close()

In [2]:
#### Diffusionnet nopt: triangle intersection losses
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "nonzero", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', "cone2", "halfbunny"]
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1, 1]
stitchschedule = [None, 'linear', 'constant']
inits = ['tutte', 'isometric']
ckptdir = "/net/scratch/rliu/NJFWand/outputs/neuralopt/softpoisson/cone_eweight0_stitchNone/ckpt"

f = open("./slurm/intersectiontests", 'w')
for data in datadirs:
    for init in inits:
        expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
        expstr += f" --softpoisson edges --arch diffusionnet --init {init} --intersectionloss"
        expstr += f" --lossdistortion arap --arapnorm --edgecut_weight 0.1 --edgecut_weight_max 0.1 --edgecut_weight_min 0 --ninit 1"
        expname = f"{data}_init{init}"

        expstr += f" --expname {expname}"
        f.write(expstr + "\n")
f.close()

In [1]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "nonzero", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', "cone2", "halfbunny"]
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1, 1]
stitchschedule = [None, 'linear', 'constant']
inits = ['tutte', 'isometric']
ckptdir = "/net/scratch/rliu/NJFWand/outputs/neuralopt/softpoisson/cone_eweight0_stitchNone/ckpt"

f = open("./slurm/cone_stitchsched", 'w')
for data in datadirs:
    for init in inits:
        for schedule in stitchschedule:
            for eweight in edgeweights:
                if eweight == 0 and schedule is not None:
                    continue
                if schedule == 'constant' and init == 'isometric':
                    continue
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                expstr += f" --softpoisson edges --arch diffusionnet --init {init}"
                expstr += f" --lossdistortion arap --arapnorm --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0 --ninit 1"

                if schedule:
                    expstr += f" --stitchschedule {schedule}"
                if schedule == "constant":
                    expstr += f" --stitchschedule_constant 0.5"

                expname = f"{data}_init{init}_eweight{eweight}_stitch{schedule}"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
    f.close()

In [11]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone2']
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1, 1]
stitchschedule = [None, 'linear', 'constant']
inits = ['tutte', 'isometric']
ckptdir = "/net/scratch/rliu/NJFWand/outputs/neuralopt/softpoisson/cone_eweight0_stitchNone/ckpt"

f = open("./slurm/cone2_stitchsched", 'w')
for data in datadirs:
    for init in inits:
        for schedule in stitchschedule:
            for eweight in edgeweights:
                if eweight == 0 and schedule is not None:
                    continue
                if schedule == 'constant' and init == 'isometric':
                    continue
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                expstr += f" --spweight softmax --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet --init {init}"
                expstr += f" --lossdistortion arap --arapnorm --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0 --ninit 1"

                if schedule:
                    expstr += f" --stitchschedule {schedule}"
                if schedule == "constant":
                    expstr += f" --stitchschedule_constant 0.5"

                expname = f"softmax_{data}_init{init}_eweight{eweight}_stitch{schedule}"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
    f.close()

In [12]:
#### Diffusionnet nopt: hyperparameter experiments for balancing cuts vs distortion optimization
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt", "--cuteps", "5e-2",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone', 'cone2']
distortionlosses = ['arap']
edgeweights = [0, 0.01, 0.1]
inits = ['isometric']

f = open("./slurm/cone_isotests", 'w')
for data in datadirs:
    for init in inits:
        for eweight in edgeweights:
            for i in range(5):
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet --init {init} --initrot"
                expstr += f" --lossdistortion arap --arapnorm --edgecut_weight {eweight} --edgecut_weight_max {eweight} --edgecut_weight_min 0 --ninit 1"
                expname = f"{i}_{data}_init{init}_eweight{eweight}"

                expstr += f" --expname {expname}"
                f.write(expstr + "\n")
f.close()

In [10]:
#### Diffusionnet cut learning with preset inits
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "25000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005",
            "--initweightinput", "--initjinput",
              "--projectname", "cone_learn", "--cuteps", "5e-2",
              ]
gradbatches = [1, 3, 8]
softmaxs = [False, True]
inits = ['tutte', 'isometric']
datas = ['cone2']
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

f = open("./slurm/conetest_ninit", 'w')
for data in datas:
    for removecut in [False]:
        for softmax in softmaxs:
            for batch in gradbatches:
                for init in inits:
                    for ninit in [-1]:
                        expstr = basestr + f" --init {init} --ninit {ninit} --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                        expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
                        expstr += f" --min_cuts 4 --max_cuts 18 --simplecut --arapnorm --edgecut_weight 0.01"
                        expstr += f" --accumulate_grad_batches {batch}"
                        expname = f"{data}_gradbatch{batch}_softmax{softmax}_init{init}"

                        if init == 'isometric':
                            expstr += " --basistype rot"

                        if removecut:
                            expstr += f" --removecutfromloss"
                            expname += f"_removecut"
                        if softmax:
                            expstr += f" --spweight softmax"
                        else:
                            expstr += f" --spweight sigmoid"
                        expstr += f" --expname {expname}"
                        f.write(expstr + "\n")
f.close()

In [3]:
#### Diffusionnet nopt experiments with preset edge cutting
# Don't remove the cut edges from the loss now
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
            "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005",
              "--spweight", "sigmoid", "--initweightinput", "--initjinput",
              "--projectname", "conetest_nopt"
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)
ignore_edges = [298, 464, 555, 301, 304, 605, 456, 46,717,552,700,699,692, 691,
                647, 190, 16, 200, 761, 757, 342, 662, 577, 122, 510, 79, 20, 193]

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cone']
ninits = ["tutte"]
removecuts = [True, False]
invjlosses = [True, False]
arapnorms = [True, False]
distortionlosses = ['arap']

f = open("./slurm/conetest_nopt", 'w')
for arapnorm in arapnorms:
    for removecut in removecuts:
        for init in ninits:
            for data in datadirs:
                for invjloss in invjlosses:
                    for i in range(4):
                        expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
                        expstr += f" --softpoisson edges --sparsecutsloss --sparsecuts_weight 0.1 --arch diffusionnet"
                        expstr += f" --lossdistortion arap"

                        if i > 0:
                            idx = (len(ignore_edges)) // 3 * i + 1
                            expstr += f" --ignorei {idx}"

                        expname = f"{data}"

                        if arapnorm:
                            expstr += f" --arapnorm"
                            expname += "_arapnorm"

                        if init == 'isometric' and not removecut:
                            continue
                        elif init == "isometric":
                            expstr += f" --init isometric --ninit 0"
                            expname += f"_initiso{i}"
                        elif init == "tutte" and i == 0:
                            expstr += f" --init tutte --ninit 0"
                            expname += f"_inittutte{i}"
                        elif init == "tutte":
                            expstr += f" --init tutte --ninit 1"
                            expname += f"_inittutte{i}"

                        if removecut:
                            expstr += f" --removecutfromloss"
                            expname += f"_removecut"

                        if invjloss:
                            expstr += " --invjloss"
                            expname += f"_invjloss"

                        expstr += f" --expname {expname}"
                        f.write(expstr + "\n")
f.close()

In [2]:
#### Neuralopt experiments with softpoisson
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--lossdistortion", "arap", "--projectname", "sp_neuralopt", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--stitchingloss", "edgecutloss", "--accumulate_grad_batches", "1", "--outputdir", "./outputs/neuralopt/softpoisson",
              "--identity", "--facedim", "3", "--seamlessedgecut", "--seamlessdelta", "0.005", "--edgecut_weight", "1",
              "--spweight", "cosine", "--initweightinput", "--initjinput",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/neuralopt/softpoisson"
datadirs = ['cylinder_nocut', 'cone', 'floatie', 'halfbunny', 'tetrahedron_sub1', 'tetrahedron_sub2']
softpoissons = ['edges']
inits = ['isometric']
stitchweights = ["softweightdetach", "stitchloss"]
sweight_schedule = ['linear', 'cosine']
spweights = ['cosine']
sparsepoisson = [True, False]

f = open("./slurm/softpoisson_nopt", 'w')
for data in datadirs:
    expstr = basestr + f" --init isometric --ninit 1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
    expstr += f" --softpoisson edges"

    expname = f"neuralopt_{data}_initinputs"
    expstr += f" --expname {expname}"
    f.write(expstr + "\n")
f.close()

In [1]:
#### Learning experiments with diffusionnet sp
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--noencoder",
              "--projectname", "softpoisson", "--lr", "1e-4",
              "--targets_per_batch", "16", "--workers", "0", "--valrenderratio", "1",
              "--accumulate_grad_batches", "1", "--outputdir", "./outputs/learning/softpoisson",
              "--initweightinput", "--initjinput", "--facedim", "128",
              "--identity", "--seamlessedgecut", "--seamlessdelta", "0.005", "--edgecut_weight", "1",
              ]
basestr = "python ./training_scripts/train.py " + " ".join(basestr)

baseoutput = "./outputs/learning/softpoisson"
datadirs = ['cone']
softpoissons = ['edges']
inits = ['tutte', 'slim', 'isometric']

f = open("./slurm/softpoisson_simple_gt", 'w')
for data in datadirs:
    for poisson in softpoissons:
        for init in inits:
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --data_file {data}.json"
            expstr += f" --softpoisson {poisson} --simplecut --arch diffusionnet --spweight sigmoid"
            expstr += " --gtuvloss"
            # expstr += " --stitchingloss edgecutloss --lossdistortion arap --sparsecutsloss"

            if data == "floatie":
                expstr += f" --min_cuts 2 --max_cuts 20"
            elif data == "cone":
                expstr += f" --min_cuts 0 --max_cuts 12"
            elif data == "tetrahedron_sub2":
                expstr += f" --min_cuts 0 --max_cuts 2 --top_k_eig 8 "
            else:
                expstr += f" --min_cuts 0 --max_cuts 20"

            if init == "slim":
                expstr += f" --slimiters 2"
            elif init == "isometric":
                expstr += f" --basistype rot"

            expname = f"gtcutlearn_{data}_init{init}"
            expstr += f" --expname {expname}"
            f.write(expstr + "\n")
f.close()

In [1]:
import os

### Soft poisson direct opt
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron4.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
stitchweight = ['softweightdetach']
distortionlosses = ['symmetricdirichlet', 'arap']
edgecuts = ['seamless', 'vertexsep']
edgeweights = ['seploss', 'softweightdetach']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs:
    for distortionloss in distortionlosses:
        for edgecut in edgecuts:
            for edgeweight in edgeweights:
                objname = os.path.basename(data).split(".")[0]
                savename = f"{objname}_ecut{edgecut}_eweight{edgeweight}_dist{distortionloss}"
                expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --savename {savename} --lr 1e-4"
                expstr += f" --distortionloss {distortionloss} --edgeweight {edgeweight} --edgecutloss {edgecut}"
                expstr += f" --seplossdelta 0.01 --stitchweight softweightdetach "
                expstr += f" --vertexseptype l2 --vertexsepsqrt"
            f.write(expstr + "\n")
f.close()

In [1]:
import os

### Soft poisson direct opt
basestr = "python opt_softpoisson.py "
datadirs = ['./data/tetrahedron4.obj', './data/tetrahedron5.obj', "./data/tetrahedron_sub1.obj", "./data/tetrahedron_sub2.obj"]
stitchweight = ['softweight', 'softweightdetach', 'seploss']
seamlesses = [True, False]
vseptype = ['l2sqrt']

f = open("slurm/softpoisson_opt", 'w')
for data in datadirs:
    for seamless in seamlesses:
        for sweight in stitchweight:
            objname = os.path.basename(data).split(".")[0]
            savename = f"{objname}_edgecut_seamless{seamless}_sweight{sweight}"
            expstr = basestr + f" --objdir {data} --niters 300000 --init isometric --vertexseploss --symmetricdirichletloss --savename {savename} --lr 1e-4"
            expstr += f" --seplossdelta 0.01"
            expstr += f" --vertexseptype l2 --vertexsepsqrt --edgecutloss seamless --stitchweight {sweight}"

            if seamless:
                expstr += f" --seamless"

            if sweight == "seploss":
                expstr += f" --niters 1000000 --lr 1e-3"

            f.write(expstr + "\n")
f.close()